In [79]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
pd.options.display.max_colwidth = 500

In [86]:
basedir = "C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/oaei/OAEI_w2v_steps_walklength1_dim100_2019_06_18_03_43_32_631149/"

gs = pd.read_csv(basedir+"oaei_gold_standard5best.csv", encoding="UTF-8", sep="\t", header=None)
gs.columns = ['src_id','tgt_id']
embs = pd.read_csv(basedir+"stratified_embeddings.csv", encoding="UTF-8", sep=",")
embs = embs[[col for col in embs.columns if re.match('x\d+', col) is not None]+['label']]
embs.columns = ["src_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('x\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['src_id'], right_on=['label'])
embs.columns = ["tgt_" + str(col) for col in [re.search("\d+", col).group(0) for col in embs.columns if re.match('src_\d+', col) is not None]] + ['label']
gs = gs.merge(embs, left_on=['tgt_id'], right_on=['label'])
gs.head()

,src_id,tgt_id,src_0,src_1,src_2,src_3,src_4,src_5,src_6,src_7,...,tgt_11,tgt_12,tgt_13,tgt_14,tgt_15,tgt_16,tgt_17,tgt_18,tgt_19,label_y
0,http://dbkwik.webdatacommons.org/darkscape/resource/sweetcorn_seed,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn_seed,0.621755,0.210856,-1.417335,0.777930,0.255901,0.800618,-0.639755,1.397252,...,-0.519015,-0.752442,0.169198,1.900312,0.682599,-0.473278,0.868871,-0.100651,0.232102,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn_seed
1,http://dbkwik.webdatacommons.org/darkscape/resource/sweetcorn_(bowl),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn_seed,0.478267,-0.917111,-0.443966,0.804700,0.789262,-0.461292,-1.199456,1.688774,...,-0.519015,-0.752442,0.169198,1.900312,0.682599,-0.473278,0.868871,-0.100651,0.232102,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn_seed
2,http://dbkwik.webdatacommons.org/darkscape/resource/sweetcorn_seed,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn,0.621755,0.210856,-1.417335,0.777930,0.255901,0.800618,-0.639755,1.397252,...,-1.444761,-0.830069,0.453246,1.086921,0.901700,-0.141183,1.148374,-0.145147,0.228291,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn
3,http://dbkwik.webdatacommons.org/darkscape/resource/burnt_sweetcorn,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn,0.339311,-0.913869,0.112679,1.115076,0.627889,0.096354,-0.953945,1.579770,...,-1.444761,-0.830069,0.453246,1.086921,0.901700,-0.141183,1.148374,-0.145147,0.228291,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn
4,http://dbkwik.webdatacommons.org/darkscape/resource/sweetcorn,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn,0.610215,0.120380,-1.097623,0.792373,0.508598,0.696596,-0.740600,1.256574,...,-1.444761,-0.830069,0.453246,1.086921,0.901700,-0.141183,1.148374,-0.145147,0.228291,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/sweetcorn


In [87]:
from gensim.models import Doc2Vec, Word2Vec
model = Word2Vec.load(basedir+"w2v.model")
def get_training_material(nid):
            res = list()
            with open(basedir+"w2v_training_material.csv", mode="r", encoding="UTF-8") as f:
                for line in f:
                    if nodeid in line.split(" "):
                        for w in line.split(" "):
                            yield w

In [88]:
nodeid="http://dbkwik.webdatacommons.org/darkscape/resource/adamant_shield_(h3)"
possible_matches_for_nodeid = gs.loc[gs.src_id==nodeid]
possible_matches_for_nodeid

,src_id,tgt_id,src_0,src_1,src_2,src_3,src_4,src_5,src_6,src_7,...,tgt_11,tgt_12,tgt_13,tgt_14,tgt_15,tgt_16,tgt_17,tgt_18,tgt_19,label_y
5833,http://dbkwik.webdatacommons.org/darkscape/resource/adamant_shield_(h3),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h5),0.322762,-0.545766,-0.611111,0.813321,1.863385,-1.040113,-0.254961,-0.030035,...,-2.300975,-0.340735,-0.102400,0.849477,0.996517,0.391566,0.271559,0.130127,1.742762,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h5)
5838,http://dbkwik.webdatacommons.org/darkscape/resource/adamant_shield_(h3),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h4),0.322762,-0.545766,-0.611111,0.813321,1.863385,-1.040113,-0.254961,-0.030035,...,-2.056457,-0.438322,-0.047396,0.894022,1.157312,0.226971,0.610822,-0.019753,1.788217,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h4)
5843,http://dbkwik.webdatacommons.org/darkscape/resource/adamant_shield_(h3),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h3),0.322762,-0.545766,-0.611111,0.813321,1.863385,-1.040113,-0.254961,-0.030035,...,-1.958787,-0.386472,0.051407,0.837814,0.953557,0.187345,0.240402,0.120411,1.967294,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h3)
5850,http://dbkwik.webdatacommons.org/darkscape/resource/adamant_shield_(h3),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h2),0.322762,-0.545766,-0.611111,0.813321,1.863385,-1.040113,-0.254961,-0.030035,...,-1.806752,-0.195634,0.242424,0.526742,0.641329,-0.295918,-0.003036,0.039415,1.480437,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h2)
5855,http://dbkwik.webdatacommons.org/darkscape/resource/adamant_shield_(h3),http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h1),0.322762,-0.545766,-0.611111,0.813321,1.863385,-1.040113,-0.254961,-0.030035,...,-1.965821,-0.502740,-0.164192,0.736181,0.979511,0.391570,0.600060,-0.041791,1.561014,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h1)


In [90]:
o1 = get_training_material(nodeid)
inverted_dict = dict()
for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
    inverted_dict[tgt_id] = [tgt_id]
    for word in get_training_material(tgt_id):
        if word in inverted_dict.keys():
            inverted_dict[word].append(tgt_id)
        else:
            inverted_dict[word] = [tgt_id]

tgt_scores = dict()
for tgt_id in possible_matches_for_nodeid.tgt_id.to_list():
    tgt_scores[tgt_id]=0
    
for tupl in model.predict_output_word(o1, topn=99999999999999):
    if tupl[0] in inverted_dict.keys():
        for tgt_id in inverted_dict[tupl[0]]:
            tgt_scores[tgt_id] = tgt_scores[tgt_id] + tupl[1]
for item in sorted(tgt_scores.items(), key = lambda kv:(kv[1], kv[0]), reverse=True):
    print(item)

('http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h1)', 13.85804412118461)
('http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h5)', 13.858042082244145)
('http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h4)', 13.85804206284019)
('http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h3)', 13.858041616856616)
('http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h2)', 13.858041383340497)


http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h1)
http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h5)
http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h4)
http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h3)
http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/adamant_shield_(h2)
